### CoreBx_multi - Process several subareas on N. Core Banks

Interpolate the North Core Banks DEMs onto rotated 1-m grids for nine regions and save each as a .nc file.

TODO: The alongshore/cross-shore names are switched.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# from dask.distributed import LocalCluster
from scipy import interpolate, signal
%matplotlib inline



In [2]:
# define all of the functions by runnng this python file
%run -i CoreBx_funcs

In [11]:
def make_grid(name=None,e0=None,n0=None,xlen=None,ylen=None,dxdy=None,theta=None):
    nx = int((1./dxdy)*xlen)
    ny = int((1./dxdy)*ylen)

    xcoords = np.linspace(0.5*dxdy,xlen-0.5*dxdy,nx)
    ycoords = np.linspace(0.5*dxdy,ylen-0.5*dxdy,ny)

    # these will be the coordinates in rotated space
    xrot, yrot = np.meshgrid(xcoords, ycoords ,sparse=False, indexing='xy')

    print('Shape of xrot, yrot: ',np.shape(xrot),np.shape(yrot))
    shp = np.shape(xrot)
    xu, yu = box2UTMh(xrot.flatten(), yrot.flatten(), e0, n0, theta)
    xu=np.reshape(xu,shp)
    yu=np.reshape(yu,shp)
    # write the UTM coords of the corners to an ASCII file
    corners = np.asarray(  [[xu[0][0],yu[0][0]],\
                           [xu[0][-1],yu[0][-1]],\
                           [xu[-1][-1],yu[-1][-1]],\
                           [xu[-1][0],yu[-1][0]],\
                           [xu[0][0],yu[0][0]]])

    print(corners)
    fn = name+'.csv'
    np.savetxt(fn, corners, delimiter=",")
    return xu, yu, xrot, yrot, xcoords, ycoords

In [7]:
fdir = "C:/crs/proj/2019_DorianOBX/WayneWright_flights/"

fnames = (\
          "2019-08-30_1m_DEM_v1_crop.tif",\
          "2019-09-12-13_DEM_1m_v1_crop.tif")

titles = ([\
         "pre-Dorian",\
         "post-Dorian"])

nf = len(fnames)

# optional median-filter smoothing of original maps
smooth = False
# kernal size...this should be an odd number >= dxy/0.1
ksize = 3

In [5]:
# Make an array of dicts, where analysis region is defined by:
#  name
#  e0 - UTM Easting of origin [m]
#  n0 - UTM Northing of origin [m]
#  xlen - Length of alongshore axis [m]
#  ylen - Length of cross-shore axis [m]
#  dxdy - grid size (must be isotropic right now) [m]
#  theta - rotation CCW from x-axis [deg]

regions = [{'name':"region_1","e0": 380400.,"n0": 3858160.,"xlen": 3400.,"ylen": 600.,"dxdy": 1.,"theta": 42.},
           {'name':"region_2","e0": 383440.,"n0": 3860830.,"xlen": 3990.,"ylen": 700.,"dxdy": 1.,"theta": 41.},
           {'name':"region_3","e0": 386485.,"n0": 3863430.,"xlen": 1200.,"ylen": 700.,"dxdy": 1.,"theta": 44.},
           {'name':"region_4","e0": 387330.,"n0": 3864280.,"xlen": 4000.,"ylen": 700.,"dxdy": 1.,"theta": 45.},
           {'name':"region_5","e0": 390140.,"n0": 3867115.,"xlen": 3400.,"ylen": 700.,"dxdy": 1.,"theta": 43.75},
           {'name':"region_6","e0": 392600.,"n0": 3869450.,"xlen": 2440.,"ylen": 700.,"dxdy": 1.,"theta": 42.},
           {'name':"region_7","e0": 394430.,"n0": 3871065.,"xlen": 3220.,"ylen": 700.,"dxdy": 1.,"theta": 42.},
           {'name':"region_8","e0": 396800.,"n0": 3873240.,"xlen": 3500.,"ylen": 700.,"dxdy": 1.,"theta": 42.},
           {'name':"region_9","e0": 399400.,"n0": 3875580.,"xlen": 3300.,"ylen": 700.,"dxdy": 1.,"theta": 40.}]

In [13]:
%%time
for r in regions:
    print(r['name'])
    xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)
    ny,nx = np.shape(xu)
    print(ny,nx)
    dslist=[]
    for i in range(nf):
        fn = fdir+fnames[i]
        print(i, fn)

        # open the tif with XArray as a DataArray
        da = xr.open_rasterio(fn)

        print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
        x = da['x'].values
        y = np.flipud(da['y'].values)

        # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
        # However, make sure to squeeze before flipping
        z = np.flipud(np.squeeze(da.values))
        print(np.shape(x),np.shape(y),np.shape(z))

        if(smooth):
            # smooth with 2D running median
            zs = signal.medfilt2d(z, kernel_size=ksize)
        else:
            zs = z

        f = interpolate.RegularGridInterpolator( (y, x), zs, method='linear')   

        # Array for interpolated elevations
        zi=np.NaN*np.ones((ny,nx))

        # this is a slow iteration through all of the points, but allows us to skip ones that are outside
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

        da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
        da = da.chunk()
        dslist.append(da)

    dsa = xr.concat(dslist, dim='map')
    fn = r['name']+'.nc'
    dsa.to_netcdf(fn)

region_1
Shape of xrot, yrot:  (600, 3400) (600, 3400)
[[ 380400.03700711 3858160.70613772]
 [ 382925.98626891 3860435.08106873]
 [ 382525.1770357  3860880.22481919]
 [ 379999.2277739  3858605.84988818]
 [ 380400.03700711 3858160.70613772]]
600 3400
0 C:/crs/proj/2019_DorianOBX/WayneWright_flights/2019-08-30_1m_DEM_v1_crop.tif
(22393,) (25165,) (1, 22393, 25165)
(25165,) (22393,) (22393, 25165)
1 C:/crs/proj/2019_DorianOBX/WayneWright_flights/2019-09-12-13_DEM_1m_v1_crop.tif
(22101,) (25420,) (1, 22101, 25420)
(25420,) (22101,) (22101, 25420)
region_2
Shape of xrot, yrot:  (700, 3990) (700, 3990)
[[ 383440.04932528 3860830.7053843 ]
 [ 386450.58584078 3863447.72485095]
 [ 385992.00057952 3863975.26684752]
 [ 382981.46406401 3861358.24738088]
 [ 383440.04932528 3860830.7053843 ]]
700 3990
0 C:/crs/proj/2019_DorianOBX/WayneWright_flights/2019-08-30_1m_DEM_v1_crop.tif
(22393,) (25165,) (1, 22393, 25165)
(25165,) (22393,) (22393, 25165)
1 C:/crs/proj/2019_DorianOBX/WayneWright_flights/2019